In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
# from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import precision_score, recall_score, f1_score

app_train = pd.read_csv('application_train.csv')
app_test = pd.read_csv('application_test.csv')
bureau = pd.read_csv('bureau.csv')
bureau_balance = pd.read_csv('bureau_balance.csv')
pos_cash_balance = pd.read_csv('POS_CASH_balance.csv')
credit_card_balance = pd.read_csv('credit_card_balance.csv')
previous_application = pd.read_csv('previous_application.csv')
installments_payments = pd.read_csv('installments_payments.csv')

In [2]:
#aggregates 
app_train

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#aggregates previous.csv
i=0
useful = ['SK_ID_PREV', 'SK_ID_CURR', 'NAME_CONTRACT_TYPE',	'AMT_ANNUITY', 'AMT_CREDIT', 'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START', 'NAME_CONTRACT_STATUS', 'DAYS_DECISION',
          'DAYS_LAST_DUE','DAYS_TERMINATION']
previous_output_cols = useful + ['TOTAL_ANNUITY','TOTAL_APPLICATION']
outs = pd.DataFrame(columns=previous_output_cols)
for app in np.unique(previous_application['SK_ID_CURR']):
    if i%10000==0: print(i)
    temp = previous_application[previous_application['SK_ID_CURR']==app].sort_values(by=['DAYS_DECISION'],ascending=False).reset_index(drop=True)
    agg = pd.DataFrame({'TOTAL_ANNUITY':[temp['AMT_ANNUITY'].sum()], 'TOTAL_APPLICATION': [temp['AMT_CREDIT'].sum()]}).reset_index(drop=True)
    last_application = pd.DataFrame(temp[useful].iloc[0,:]).transpose()
    out = pd.concat([last_application, agg],axis=1)
    outs = pd.concat([outs, out],axis=0)
    i += 1

In [124]:
#installment.csv (payment)
i=0
output = pd.DataFrame(columns=["SK_ID_CURR", "TOTAL_PAYMENT", "TOTAL_OVERDUE_DAYS"])
for app in np.unique(installments_payments['SK_ID_CURR']):
    if i%10000 == 0:
        print(i)
    temp = installments_payments[installments_payments['SK_ID_CURR']==app].sort_values(by=['DAYS_INSTALMENT'],ascending=False).reset_index(drop=True).dropna()
    temp["OVERDUE_DAYS"] = temp["DAYS_ENTRY_PAYMENT"] - temp["DAYS_INSTALMENT"]
    single = pd.DataFrame({'SK_ID_CURR': [app], 'TOTAL_PAYMENT':[temp["AMT_PAYMENT"].sum()], 'TOTAL_OVERDUE_DAYS': [temp["OVERDUE_DAYS"].sum()]}).reset_index(drop=True)
    output = pd.concat([output, single],axis=0)
    i+=1
output


0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000


,SK_ID_CURR,TOTAL_PAYMENT,TOTAL_OVERDUE_DAYS
0,100001,41195.925,-51.0
0,100002,219625.695,-388.0
0,100003,1618864.650,-179.0
0,100004,21288.465,-23.0
0,100005,56161.845,-212.0
...,...,...,...
0,456251,52450.470,-254.0
0,456252,60419.205,-17.0
0,456253,57622.815,-203.0
0,456254,194556.825,-361.0


In [153]:
#credit_card_balance
# cred = credit_card_balance[credit_card_balance["SK_ID_CURR"]==126868]
# cred[cred["SK_DPD"]!=0]

i=0
output_cred = pd.DataFrame(columns=['SK_ID_CURR','TOTAL_BALANCE','TOTAL_LIMIT','AVG_DIFF_RECEIV_BALANCE','AVG_RATIO_CASH'])
credit_card_balance_filled = credit_card_balance.fillna(0)
for app in np.unique(credit_card_balance_filled['SK_ID_CURR']):
    if i%10000==0: print(i)
    temp = credit_card_balance_filled[credit_card_balance_filled['SK_ID_CURR']==app]
    temp = temp[temp['MONTHS_BALANCE']>=-6].sort_values(by=['MONTHS_BALANCE'],ascending=False).reset_index(drop=True)
    #one-line aggregation
    agg = pd.DataFrame({'SK_ID_CURR':[app], 'TOTAL_BALANCE':[temp['AMT_BALANCE'].sum()],'TOTAL_LIMIT':[temp['AMT_CREDIT_LIMIT_ACTUAL'].sum()],
                        'AVG_DIFF_RECEIV_BALANCE':[(temp['AMT_TOTAL_RECEIVABLE'] - temp['AMT_CREDIT_LIMIT_ACTUAL']).mean()],
                        'AVG_RATIO_CASH':[(temp["AMT_DRAWINGS_ATM_CURRENT"]/(temp["AMT_DRAWINGS_CURRENT"] + 0.0001)).mean()]})
    output_cred = pd.concat([output_cred, agg],axis=0)
    i += 1
output_cred


0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000


,SK_ID_CURR,TOTAL_BALANCE,TOTAL_LIMIT,AVG_DIFF_RECEIV_BALANCE,AVG_RATIO_CASH
0,100006,0.00,1620000,-270000.000,0.0
0,100011,0.00,450000,-90000.000,0.0
0,100013,0.00,270000,-45000.000,0.0
0,100021,0.00,3375000,-675000.000,0.0
0,100023,0.00,675000,-225000.000,0.0
...,...,...,...,...,...
0,456244,0.00,0,0.000,0.0
0,456246,44996.94,675000,-126000.711,0.0
0,456247,0.00,0,0.000,0.0
0,456248,0.00,4500000,-900000.000,0.0


In [157]:
#save to csv
output.to_csv("installments_aggregates.csv", index=False)
output_cred.to_csv("credit_aggregates.csv", index=False)
outs.to_csv("previous_aggregates.csv",index=False)

In [96]:
#test
useful = ['SK_ID_PREV', 'SK_ID_CURR', 'NAME_CONTRACT_TYPE',	'AMT_ANNUITY', 'AMT_CREDIT', 'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START', 'NAME_CONTRACT_STATUS', 'DAYS_DECISION',
          'DAYS_LAST_DUE','DAYS_TERMINATION']
temp = previous_application[previous_application['SK_ID_CURR']==204857].sort_values(by=['DAYS_DECISION'],ascending=False).reset_index(drop=True)
agg = pd.DataFrame({'TOTAL_ANNUITY':[temp['AMT_ANNUITY'].sum()], 'TOTAL_APPLICATION': [temp['AMT_CREDIT'].sum()]}).reset_index(drop=True)
agg
last_application = pd.DataFrame(temp[useful].iloc[0,:]).transpose()
last_application
pd.concat([last_application, agg],axis=1)



,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_CREDIT,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,NAME_CONTRACT_STATUS,DAYS_DECISION,DAYS_LAST_DUE,DAYS_TERMINATION,TOTAL_ANNUITY,TOTAL_APPLICATION
0,1875783,204857,Consumer loans,9323.865,80509.5,SUNDAY,10,Approved,-136,365243.0,365243.0,21127.995,166482.0


In [ ]:
bureau.head(10)
bureau.loc[bureau['SK_ID_CURR']==100004]

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
363104,100004,6829133,Closed,currency 1,-1326,0,-595.0,-683.0,NaN,0,94500.0,0.0,0.0,0.0,Consumer credit,-682,NaN
363105,100004,6829134,Closed,currency 1,-408,0,-382.0,-382.0,0.0,0,94537.8,0.0,0.0,0.0,Consumer credit,-382,NaN


In [ ]:
pos_cash_balance.head(10)

In [ ]:
#检查prev ID是不是和CURR ID在一个文件内，他们代表同一个人
#结论：prev ID存在于previous.csv
#每一个现在的神情记录之前都有记录？？？
pos_cash_balance["SK_ID_PREV"].isnull().sum()
print(np.unique(pos_cash_balance["SK_ID_CURR"]).shape)
#group by之后
# pos_cash_balance.groupby(['SK_ID_CURR']).head(50)
#curr pos 记录是否对应多个prev pos记录?
np.unique(pos_cash_balance["SK_ID_CURR"]).shape[0] - np.unique(pos_cash_balance["SK_ID_PREV"]).shape[0]
#多了59万个
reps = []
print(pos_cash_balance.shape[0])
for i in range(0, 2000000):
    if pos_cash_balance.iloc[i, 1] == pos_cash_balance.iloc[i-1, 1]: 
        reps.append(i-1)
        reps.append(i)
reps
#reps = [(i,i+1) for i in range(pos_cash_balance.shape[0]) if pos_cash_balance.iloc[i, 1] == pos_cash_balance.iloc[i+1, 1]]



(337252,)
10001358


[23465,
 23466,
 89604,
 89605,
 196218,
 196219,
 704341,
 704342,
 1239803,
 1239804,
 1264209,
 1264210,
 1283288,
 1283289,
 1407087,
 1407088,
 1470118,
 1470119,
 1631045,
 1631046,
 1739774,
 1739775,
 1746266,
 1746267]

In [43]:
#previous
previous_application
#为什么previous的行数这么多？？？160万？
print(np.unique(previous_application["SK_ID_PREV"]).shape)
#一个人有多个previous app ID意味着什么？这个人有11次申请记录：
previous_application[previous_application['SK_ID_CURR']==274851]

(1670214,)


,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
170832,2671009,274851,Revolving loans,2250.000,45000.0,45000.0,NaN,45000.0,SATURDAY,13,...,Consumer electronics,0.0,XNA,Card Street,365243.0,365243.0,365243.0,365243.0,365243.0,0.0
270796,2068787,274851,Cash loans,NaN,0.0,0.0,NaN,NaN,THURSDAY,11,...,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN
365136,1402532,274851,Cash loans,12178.125,90000.0,109156.5,NaN,90000.0,MONDAY,10,...,XNA,12.0,high,Cash X-Sell: high,NaN,NaN,NaN,NaN,NaN,NaN
518355,2256420,274851,Cash loans,15883.335,135000.0,143910.0,0.0,135000.0,TUESDAY,9,...,Consumer electronics,12.0,high,Cash Street: high,365243.0,-1931.0,-1601.0,-1631.0,-1627.0,1.0
825798,2371489,274851,Cash loans,9765.990,90000.0,121968.0,NaN,90000.0,FRIDAY,10,...,XNA,24.0,high,Cash Street: high,365243.0,-1200.0,-510.0,-510.0,-500.0,1.0
856320,2717111,274851,Consumer loans,11518.020,53055.0,56488.5,5305.5,53055.0,WEDNESDAY,12,...,Consumer electronics,6.0,high,POS household with interest,365243.0,-1838.0,-1688.0,-1688.0,-1683.0,0.0
924697,1386886,274851,Consumer loans,5387.085,37305.0,40383.0,0.0,37305.0,THURSDAY,16,...,Connectivity,10.0,high,POS mobile with interest,365243.0,-2243.0,-1973.0,-2093.0,-2082.0,1.0
1113642,2429282,274851,Cash loans,16341.165,112500.0,137691.0,NaN,112500.0,FRIDAY,15,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-367.0,-37.0,-37.0,-31.0,1.0
1206363,1515702,274851,Cash loans,14910.930,135000.0,172206.0,0.0,135000.0,FRIDAY,11,...,XNA,18.0,high,Cash X-Sell: high,365243.0,-1606.0,-1096.0,-1096.0,-1088.0,1.0
1447877,2240886,274851,Cash loans,14418.090,130500.0,166464.0,0.0,130500.0,MONDAY,11,...,XNA,18.0,high,Cash X-Sell: high,365243.0,-1554.0,-1044.0,-1044.0,-1039.0,1.0


In [ ]:


app = app_train.append(app_test).reset_index()
bureau = bureau.merge(bureau_balance, on='SK_ID_BUREAU', how='left')
pos_cash_balance = pos_cash_balance.merge(previous_application, on='SK_ID_PREV', how='left')
credit_card_balance = credit_card_balance.merge(previous_application, on='SK_ID_PREV', how='left')
installments_payments = installments_payments.merge(previous_application, on='SK_ID_PREV', how='left')

merged_data = app.merge(bureau, on='SK_ID_CURR', how='left')
merged_data = merged_data.merge(pos_cash_balance, on='SK_ID_CURR', how='left')
merged_data = merged_data.merge(credit_card_balance, on='SK_ID_CURR', how='left')
merged_data = merged_data.merge(installments_payments, on='SK_ID_CURR', how='left')

merged_data.drop(['SK_ID_CURR', 'SK_ID_PREV', 'SK_ID_BUREAU'], axis=1, inplace=True)

num_features = [...]  
cat_features = [...]
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_features),
        ('cat', cat_transformer, cat_features)
    ])

# Prepare data for training and validation
X = merged_data.drop('TARGET', axis=1)
y = merged_data['TARGET'].dropna()

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


models = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'XGBoost': XGBClassifier(random_state=42),
    'Neural Network': MLPClassifier(hidden_layer_sizes=(100,), random_state=42)
}


for name, model in models.items():
    print(f"Training {name}...")
    pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', model)])
    pipeline.fit(X_train, y_train)
    
    y_pred = pipeline.predict(X_val)
    
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)
    
    print(f"{name} Results:")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print()

